Library

In [1]:
from pathlib import Path
import face_recognition
import pickle
from collections import Counter
from PIL import Image, ImageDraw
import cv2
import sys
from time import sleep
import random
import string
import pandas as pd
import numpy as np
from sklearn.svm import SVC

Set file path

In [2]:
DEFAULT_ENCODING_PATH = Path("output/encodings.pkl")
BOUNDING_BOX_COLOR = "green"
TEXT_COLOR = "white"

Path("training").mkdir(exist_ok=True)
Path("output").mkdir(exist_ok=True)
Path("validation").mkdir(exist_ok=True)
Path("img_out").mkdir(exist_ok=True)

Fungsi untuk encoding wajah menggunakan histogram of oriented gradients

In [3]:
def encode_known_faces(
    model: str = "hog", encodings_location: Path = DEFAULT_ENCODING_PATH
) -> None:
    names = []
    encodings = []
    for filepath in Path("training").glob("*/*"):
        name = filepath.parent.name
        image = face_recognition.load_image_file(filepath)

        face_locations = face_recognition.face_locations(image, model=model)
        face_encodings = face_recognition.face_encodings(image, face_locations)

        for encoding in face_encodings:
            names.append(name)
            encodings.append(encoding)

    name_encodings = {"names": names, "encodings": encodings}
    with encodings_location.open(mode="wb") as f:
        pickle.dump(name_encodings, f)

Fungsi untuk mengambil gambar menggunakan webcam

In [4]:
def img_capture():
    key = cv2.waitKey(1)
    webcam = cv2.VideoCapture(0)
    sleep(2)
    while True:

        try:
            check, frame = webcam.read()
        
            cv2.imshow("Capturing", frame)
            key = cv2.waitKey(1)
            if key == ord('s'): 
                cv2.imwrite(filename='identify.jpg', img=frame)
                webcam.release()
                cv2.destroyAllWindows()                             
                break
        
            elif key == ord('q'):
                webcam.release()
                cv2.destroyAllWindows()
                break
    
        except(KeyboardInterrupt):
            print("Turning camera off")
            webcam.release()        
            cv2.destroyAllWindows()
            break

Memanggil fungsi encoding wajah

In [5]:
encode_known_faces()

Membuat list nama

In [6]:
name_list = []
unique_name_list = []

for filepath in Path("training").glob("*/*"):
    name_list.append(filepath.parent.name)

for i in range(len(name_list)):
    if name_list[i] not in unique_name_list:
        unique_name_list.append(name_list[i])
    else: continue

Fungsi konversi data encoding wajah untuk training SVM

In [7]:
def train_svm():  
    f = DEFAULT_ENCODING_PATH.open(mode = 'rb')

    df_known = pd.DataFrame(pickle.load(f))

    df_encodings = pd.DataFrame(df_known['encodings'])

    encodings_list = [[ df_encodings['encodings'][i][j] for i in range(len(df_encodings['encodings']))] for j in range(len(df_encodings['encodings'][0]))]

    df_encodings_list = pd.DataFrame(encodings_list)

    df_enc = df_encodings_list.transpose()

    target = []

    for i in range(len(df_known['names'])):
        for j in range(len(unique_name_list)):
            if df_known['names'][i] == unique_name_list[j]:
                target.append(j)
            else: continue

    array_target = np.array(target)

    model_svm = SVC()
    model_svm.fit(df_enc, array_target)

    return model_svm

Memanggil fungsi training SVM dan simpan hasilnya

In [8]:
model_svm = train_svm()

Fungsi untuk mengenali wajah menggunakan SVM

In [28]:
def recognize_face(
    image_location: str,
    model: str = "hog",
    encodings_location: Path = DEFAULT_ENCODING_PATH,
) -> None:
    with encodings_location.open(mode="rb") as f:
        loaded_encodings = pickle.load(f)

    name = str

    input_image = face_recognition.load_image_file(image_location)


    input_face_locations = face_recognition.face_locations(
        input_image, model=model
    )
    input_face_encodings = face_recognition.face_encodings(
        input_image, input_face_locations
    )

    input_enc = [input_face_encodings[0][i] for i in range(len(input_face_encodings[0]))]
    
    df_input_enc = pd.DataFrame(input_enc).transpose()

    pillow_image = Image.fromarray(input_image)
    draw = ImageDraw.Draw(pillow_image)

    for bounding_box, unknown_encoding in zip(
        input_face_locations, input_face_encodings
    ):
        name = unique_name_list[_recognize_face(df_input_enc)[0]]
        if not name:
            name = "Unknown"
        _display_face(draw, bounding_box, name)

    del draw
    rand = ''.join((random.choice(string.ascii_letters) for x in range(8)))
    pillow_image.save("img_out/" + rand + ".jpg")
    #pillow_image.show()
    return name

Fungsi pembantu recognize_face() untuk memanggil model SVM

In [10]:
def _recognize_face(input_enc):
    id = model_svm.predict(input_enc)
    return id

Fungsi pembantu recognize_face() untuk menampilkan wajah

In [11]:
def _display_face(draw, bounding_box, name):
    top, right, bottom, left = bounding_box
    draw.rectangle(((left, top), (right, bottom)), outline=BOUNDING_BOX_COLOR)
    text_left, text_top, text_right, text_bottom = draw.textbbox(
        (left, bottom), name
    )
    draw.rectangle(
        ((text_left, text_top), (text_right, text_bottom)),
        fill="green",
        outline="green",
    )
    draw.text(
        (text_left, text_top),
        name,
        fill="white",
    )

Fungsi untuk validasi dan menghitung akurasi

In [12]:
def validate(model: str = "hog"):
    T = 0
    F = 0
    
    for filepath in Path("validation").rglob("*"):
        if filepath.is_file():
            name = recognize_face(
                image_location=str(filepath.absolute()), model=model
            )
            print(name,' | ', filepath.parent.name)
            if name == filepath.parent.name:
                T = T+1
            else: F = F+1

    accuracy = T/(T+F)*100
    print(accuracy,'%')

Memanggil fungsi validasi

In [13]:
validate()

Bernard Hugo  |  Bernard Hugo
Feelouis Elfredo  |  Feelouis Elfredo
Marbella  |  Marbella
Mario Iskandar  |  Mario Iskandar
Moethia Shakira  |  Moethia Shakira
Nicholas Javier  |  Nicholas Javier
Raphael Alexander Lesmana  |  Raphael Alexander Lesmana
Roger Julianto Angryawan  |  Roger Julianto Angryawan
Shelly Alfianda  |  Shelly Alfianda
Sidi Janardhana Gatra Parahita  |  Sidi Janardhana Gatra Parahita
Shelly Alfianda  |  Yennifer Wilanata
90.9090909090909 %


Fungsi untuk mengenali wajah menggunakan webcam

In [14]:
def verify():
    img_capture()
    recognize_face("identify.jpg")

Memanggil fungsi verify()

In [29]:
verify()